In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%pip install accelerate
%pip install diffusers
%pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.7/270.7 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 5.9 MB/s eta 0:00:00


In [ ]:
from dataclasses import dataclass

@dataclass
class TrainingConfig:
    image_size = 28  # the generated image resolution
    feature_dim = 10 # feature dimensionality for embeddings
    train_batch_size = 16
    eval_batch_size = 16  # how many images to sample during evaluation
    num_epochs = 10
    gradient_accumulation_steps = 1
    learning_rate = 1e-4
    lr_warmup_steps = 500
    mixed_precision = "fp16"  # `no` for float32, `fp16` for automatic mixed precision
    output_dir = "pf-mnist-28"  # the model name locally and on the HF Hub

    push_to_hub = False  # whether to upload the saved model to the HF Hub
    hub_model_id = "<your-username>/<my-awesome-model>"  # the name of the repository to create on the HF Hub
    hub_private_repo = False
    overwrite_output_dir = True  # overwrite the old model when re-running the notebook
    seed = 0


config = TrainingConfig()

In [ ]:
from datasets import load_dataset
from datasets import Dataset
import numpy as np

config.dataset_name = "mnist"
dataset = load_dataset(config.dataset_name, split="train")

N_classes = 10
N_models = 5
j = N_classes // N_models

# homogeneous datasets
# sum (model_labels[i][j] over i) = 1 for all j
model_labels = [[1/18 for _ in range(N_classes)] for _ in range(N_models)]
for i, _ in enumerate(model_labels):
    model_labels[i][i*j] = 1/4
    model_labels[i][i*j+1] = 1/4

print(model_labels)

def splitPerc(l, perc):
    splits = np.cumsum(perc)
    splits = splits[:-1]
    splits *= len(l)
    splits = splits.round().astype(np.int)
    return np.split(l, splits)

def get_datasets(examples, model_labels):
  images = {}
  for i, ex in enumerate(zip(examples)):
    ex = ex[0]
    label = ex['label']
    image = ex['image']
    if label in images:
      images[label].append(image)
    else:
      images[label] = [image]

  label_splits = [[] for _ in range(N_classes)]
  for label in images:
    images[label] = np.array(images[label])
    np.random.shuffle(images[label])
    split = np.array([model_labels[i][label] for i in range(N_models)])
    label_splits[label] = splitPerc(images[label], split)

  datasets = [None for _ in range(N_models)]
  for i in range(N_models):
    model_images = []
    model_labels = []
    for c in range(N_classes):
      model_labels += [c for _ in range(len(label_splits[c][i]))]
      model_images += label_splits[c][i].tolist()
    datasets[i] = Dataset.from_dict({'image' : model_images, 'label' :  model_labels})

  return datasets

datasets = get_datasets(dataset, model_labels)

[[0.25, 0.25, 0.05555555555555555, 0.05555555555555555, 0.05555555555555555, 0.05555555555555555, 0.05555555555555555, 0.05555555555555555, 0.05555555555555555, 0.05555555555555555], [0.05555555555555555, 0.05555555555555555, 0.25, 0.25, 0.05555555555555555, 0.05555555555555555, 0.05555555555555555, 0.05555555555555555, 0.05555555555555555, 0.05555555555555555], [0.05555555555555555, 0.05555555555555555, 0.05555555555555555, 0.05555555555555555, 0.25, 0.25, 0.05555555555555555, 0.05555555555555555, 0.05555555555555555, 0.05555555555555555], [0.05555555555555555, 0.05555555555555555, 0.05555555555555555, 0.05555555555555555, 0.05555555555555555, 0.05555555555555555, 0.25, 0.25, 0.05555555555555555, 0.05555555555555555], [0.05555555555555555, 0.05555555555555555, 0.05555555555555555, 0.05555555555555555, 0.05555555555555555, 0.05555555555555555, 0.05555555555555555, 0.05555555555555555, 0.25, 0.25]]


<ipython-input-5-ee0e72c9158f>:41: FutureWarning: The input object of type 'PngImageFile' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'PngImageFile', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  images[label] = np.array(images[label])
<ipython-input-5-ee0e72c9158f>:41: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  images[label] = np.array(images[label])
<ipython-input-5-ee0e72c9158f>:25: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To si

In [ ]:
from diffusers import  UNet2DConditionModel, UNet2DModel

def get_model():
  return UNet2DConditionModel(
      sample_size=config.image_size,  # the target image resolution
      in_channels=1,  # the number of input channels, 3 for RGB images
      out_channels=1,  # the number of output channels
      layers_per_block=2,  # how many ResNet layers to use per UNet block
    #   block_out_channels=(32, 64, 128),  # the number of output channels for each UNet block
      block_out_channels=(32, 480, 576, 576),
      cross_attention_dim=10,
    #   cross_attention_dim=10,
      down_block_types=(
          "DownBlock2D",  # a regular ResNet downsampling block
          "DownBlock2D",
          "CrossAttnDownBlock2D",  # a ResNet downsampling block with spatial self-attention
          "DownBlock2D",
      ),
      up_block_types=(
          "UpBlock2D",  # a regular ResNet upsampling block
          "UpBlock2D",
          "CrossAttnUpBlock2D",  # a ResNet upsampling block with spatial self-attention
          "UpBlock2D",
      ),
  )

models = []
for i in range(N_models):
  models.append(get_model())

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

In [ ]:
from torchvision.transforms import Compose, ToTensor, Lambda, ToPILImage, CenterCrop, Resize
from torchvision import transforms
from torch.utils.data import DataLoader

from torchvision.transforms import Compose, ToTensor, Lambda, ToPILImage, CenterCrop, Resize
from torchvision import transforms
from torch.utils.data import DataLoader

# define image transformations (e.g. using torchvision)
transform = Compose([
            transforms.ToTensor(),
])

# define function
def transforms(examples):
   examples["image"] = [transform(image.convert("L")) for i, image in enumerate(examples["image"])]
   return examples

transformed_datasets = []
for dataset in datasets:
  transformed_datasets.append(dataset.with_transform(transforms))


In [ ]:
import torch
from diffusers.optimization import get_cosine_schedule_with_warmup

train_dataloaders = [torch.utils.data.DataLoader(transformed_datasets[i], batch_size=config.train_batch_size, shuffle=True)
                    for i in range(N_models)]
optimizers = [torch.optim.AdamW(models[i].parameters(), lr=config.learning_rate)
                    for i in range(N_models)]

lr_schedulers = [get_cosine_schedule_with_warmup(
                  optimizer=optimizers[i],
                  num_warmup_steps=config.lr_warmup_steps,
                  num_training_steps=(len(train_dataloaders[i]) * config.num_epochs),
              ) for i in range(N_models)]

In [ ]:
import numpy as np
import functools

def marginal_prob_std(t, sigma):
  """Compute the mean and standard deviation of $p_{0t}(x(t) | x(0))$.

  Args:
    t: A vector of time steps.
    sigma: The $\sigma$ in our SDE.

  Returns:
    The standard deviation.
  """
  t = torch.tensor(t)
  return torch.sqrt((sigma**(2 * t) - 1.) / 2. / np.log(sigma))

def diffusion_coeff(t, sigma):
  """Compute the diffusion coefficient of our SDE.

  Args:
    t: A vector of time steps.
    sigma: The $\sigma$ in our SDE.

  Returns:
    The vector of diffusion coefficients.
  """
  return torch.tensor(sigma**t)

sigma =  25.0
marginal_prob_std_fn = functools.partial(marginal_prob_std, sigma=sigma)
diffusion_coeff_fn = functools.partial(diffusion_coeff, sigma=sigma)

In [ ]:
def loss_fn(model, x, marginal_prob_std, eps=1e-5, hs=torch.zeros(1, 1, 1280)):
    """The loss function for training score-based generative models.

    Args:
        model: A PyTorch model instance that represents a
        time-dependent score-based model.
        x: A mini-batch of training data.
        marginal_prob_std: A function that gives the standard deviation of
        the perturbation kernel.
        eps: A tolerance value for numerical stability.
    """

    random_t = torch.rand(x.shape[0], device=x.device) * (1. - eps) + eps
    z = torch.randn_like(x)
    std = marginal_prob_std(random_t)
    perturbed_x = x + z * std[:, None, None, None]
    score = model(perturbed_x, timestep=random_t, encoder_hidden_states=hs).sample
    loss = torch.mean(torch.sum((score * std[:, None, None, None] + z)**2, dim=(1,2,3)))
    return loss

In [ ]:
from diffusers.utils import make_image_grid
import os
from accelerate import Accelerator
from huggingface_hub import create_repo, upload_folder
from tqdm.auto import tqdm
from pathlib import Path
import os
import torch.nn.functional as F

def training_loop_sde(config, model, noise_scheduler, optimizer, train_dataloader, lr_scheduler):
    accelerator = Accelerator(
        mixed_precision=config.mixed_precision,
        gradient_accumulation_steps=config.gradient_accumulation_steps,
        log_with="tensorboard",
        project_dir=os.path.join(config.output_dir, "logs"),
    )

    if accelerator.is_main_process:
        if config.output_dir is not None:
            os.makedirs(config.output_dir, exist_ok=True)
        if config.push_to_hub:
            repo_id = create_repo(
                repo_id=config.hub_model_id or Path(config.output_dir).name, exist_ok=True
            ).repo_id
        accelerator.init_trackers("train_example")

    # Prepare everything
    # There is no specific order to remember, you just need to unpack the
    # objects in the same order you gave them to the prepare method.
    model, optimizer, train_dataloader, lr_scheduler = accelerator.prepare(
        model, optimizer, train_dataloader, lr_scheduler
    )

    global_step = 0
    # eps - tolerance value for numerical stability
    eps = 1e-5
    # Now you train the model
    for epoch in range(config.num_epochs):
        progress_bar = tqdm(total=len(train_dataloader), disable=not accelerator.is_local_main_process)
        progress_bar.set_description(f"Epoch {epoch}")

        for step, batch in enumerate(train_dataloader):
            clean_images = batch["image"]
            # Sample noise to add to the images
            noise = torch.randn(clean_images.shape, device=clean_images.device)
            bs = clean_images.shape[0]

            # emb = torch.zeros(bs, 1, config.feature_dim)
            emb = F.one_hot(batch["label"], num_classes=N_classes)
            emb = emb[:, None, :].to(torch.half)
            emb = emb.to('cuda:0')

            # Sample a random timestep for each image
            # timesteps = torch.randint(
            #     0, noise_scheduler.config.num_train_timesteps, (bs,), device=clean_images.device,
            #     dtype=torch.int64
            # )

            # Add noise to the clean images according to the noise magnitude at each timestep
            # (this is the forward diffusion process)
            # noisy_images = noise_scheduler.add_noise(clean_images, noise, timesteps)

            with accelerator.accumulate(model):
                # This bit is from the example notebook by Song et al. (2021)
                # z = torch.randn_like(batch["image"])
                # std = marginal_prob_std(timesteps, 0.01)
                # score = model(noisy_images, timestep=timesteps, return_dict=False)[0]
                # loss = torch.mean(torch.sum((score * std[:, None, None, None] + z)**2, dim=(1,2,3)))
                loss = loss_fn(model, clean_images, marginal_prob_std_fn, hs=emb)
                accelerator.backward(loss)

                accelerator.clip_grad_norm_(model.parameters(), 1.0)
                optimizer.step()
                lr_scheduler.step()
                optimizer.zero_grad()

            progress_bar.update(1)
            logs = {"loss": loss.detach().item(), "lr": lr_scheduler.get_last_lr()[0], "step": global_step}
            progress_bar.set_postfix(**logs)
            accelerator.log(logs, step=global_step)
            global_step += 1

In [ ]:
from accelerate import notebook_launcher
from diffusers import ScoreSdeVeScheduler
import gc
noise_scheduler = ScoreSdeVeScheduler(num_train_timesteps=1000)



for i in range(N_models):
  args = (config, models[i], noise_scheduler, optimizers[i], train_dataloaders[i], lr_schedulers[i],)
  notebook_launcher(training_loop_sde, args, num_processes=1)
  models[i].save_pretrained('/content/drive/MyDrive/Control_model/5_model_mix/model_dig_{}'.format(i), safe_serialization=True)
  models[i].cpu()


Launching training on one GPU.


  0%|          | 0/363 [00:00<?, ?it/s]

<ipython-input-9-8f500bc53f7d>:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  t = torch.tensor(t)


  0%|          | 0/363 [00:00<?, ?it/s]

  0%|          | 0/363 [00:00<?, ?it/s]

...

  0%|          | 0/2331 [00:00<?, ?it/s]

  0%|          | 0/2331 [00:00<?, ?it/s]

  0%|          | 0/2331 [00:00<?, ?it/s]

  0%|          | 0/2331 [00:00<?, ?it/s]

  0%|          | 0/2331 [00:00<?, ?it/s]

In [ ]:
!zip -r ../model.zip ../model/

updating: ../model/ (stored 0%)
  adding: ../model/model_dig_9/ (stored 0%)
  adding: ../model/model_dig_9/diffusion_pytorch_model.safetensors (deflated 8%)
  adding: ../model/model_dig_9/config.json (deflated 65%)
  adding: ../model/model_dig_7/ (stored 0%)
  adding: ../model/model_dig_7/diffusion_pytorch_model.safetensors (deflated 8%)
  adding: ../model/model_dig_7/config.json (deflated 65%)
  adding: ../model/model_dig_1/ (stored 0%)
  adding: ../model/model_dig_1/diffusion_pytorch_model.safetensors (deflated 8%)
  adding: ../model/model_dig_1/config.json (deflated 65%)
  adding: ../model/model_dig_6/ (stored 0%)
  adding: ../model/model_dig_6/diffusion_pytorch_model.safetensors (deflated 8%)
  adding: ../model/model_dig_6/config.json (deflated 65%)
  adding: ../model/model_dig_2/ (stored 0%)
  adding: ../model/model_dig_2/diffusion_pytorch_model.safetensors (deflated 8%)
  adding: ../model/model_dig_2/config.json (deflated 65%)
  adding: ../model/model_dig_4/ (stored 0%)
  adding:

In [ ]:
import torch
from PIL import Image

sample_size = config.image_size
noise = torch.randn((1, 1, sample_size, sample_size), device="cuda")
input = noise

Image.fromarray(((input.cpu().permute(0, 2, 3, 1) + 1.0) * 127.5).type(torch.uint8).numpy()[0].squeeze())

# for t in noise_scheduler.timesteps:
#     with torch.no_grad():
#         noisy_residual = model(input, t).sample
#     previous_noisy_sample = noise_scheduler.step_pred(noisy_residual, t, input).prev_sample
#     input = previous_noisy_sample

In [ ]:
#@title Define the ODE sampler (double click to expand or collapse)

from scipy import integrate

## The error tolerance for the black-box ODE solver
error_tolerance = 1e-5 #@param {'type': 'number'}
def ode_sampler(score_models,
                marginal_prob_std,
                diffusion_coeff,
                batch_size=64,
                atol=error_tolerance,
                rtol=error_tolerance,
                device='cuda',
                label=0,
                z=None,
                eps=1e-3):
  """Generate samples from score-based models with black-box ODE solvers.

  Args:
    score_model: A PyTorch model that represents the time-dependent score-based model.
    marginal_prob_std: A function that returns the standard deviation
      of the perturbation kernel.
    diffusion_coeff: A function that returns the diffusion coefficient of the SDE.
    batch_size: The number of samplers to generate by calling this function once.
    atol: Tolerance of absolute errors.
    rtol: Tolerance of relative errors.
    device: 'cuda' for running on GPUs, and 'cpu' for running on CPUs.
    z: The latent code that governs the final sample. If None, we start from p_1;
      otherwise, we start from the given z.
    eps: The smallest time step for numerical stability.
  """
  t = torch.ones(batch_size, device=device)
  # Create the latent code
  if z is None:
    init_x = torch.randn(batch_size, 1, 28, 28, device=device) \
      * marginal_prob_std(t)[:, None, None, None]
  else:
    init_x = z

  shape = init_x.shape
  def score_eval_wrapper(sample, time_steps):
    """A wrapper of the score-based model for use by the ODE solver."""
    sample = torch.tensor(sample, device=device, dtype=torch.float32).reshape(shape)
    time_steps = torch.tensor(time_steps, device=device, dtype=torch.float32).reshape((sample.shape[0], ))
    # emb = torch.zeros(bs, 1, config.feature_dim)
    hs = F.one_hot(torch.tensor([label]), num_classes=N_classes)
    hs = hs[:, None, :].to(torch.half)
    hs = hs.to('cuda:0')

    with torch.no_grad():
        score = sum(score_model(sample, time_steps, encoder_hidden_states=hs).sample
                    for score_model in score_models) / len(score_models)
    return score.cpu().numpy().reshape((-1,)).astype(np.float64)

  def ode_func(t, x):
    """The ODE function for use by the ODE solver."""
    time_steps = np.ones((shape[0],)) * t
    g = diffusion_coeff(torch.tensor(t)).cpu().numpy()
    return  -0.5 * (g**2) * score_eval_wrapper(x, time_steps)

  # Run the black-box ODE solver.
  res = integrate.solve_ivp(ode_func, (1., eps), init_x.reshape(-1).cpu().numpy(), rtol=rtol, atol=atol, method='RK45')
  print(f"Number of function evaluations: {res.nfev}")
  x = torch.tensor(res.y[:, -1], device=device).reshape(shape)

  return x

In [ ]:
from torchvision.utils import make_grid

## Load the pre-trained checkpoint from disk.
device = 'cuda' #@param ['cuda', 'cpu'] {'type':'string'}

sample_batch_size = 1 #@param {'type':'integer'}
sampler = ode_sampler #@param ['Euler_Maruyama_sampler', 'pc_sampler', 'ode_sampler'] {'type': 'raw'}

## Generate samples using the specified sampler.
samples = sampler(models,
                  marginal_prob_std_fn,
                  diffusion_coeff_fn,
                  sample_batch_size,
                  device=device,
                  label=2)

## Sample visualization.
samples = samples.clamp(0.0, 1.0)
%matplotlib inline
import matplotlib.pyplot as plt
sample_grid = make_grid(samples, nrow=int(np.sqrt(sample_batch_size)))

plt.figure(figsize=(6,6))
plt.axis('off')
plt.imshow(sample_grid.permute(1, 2, 0).cpu(), vmin=0., vmax=1.)
plt.show()


<ipython-input-7-8f500bc53f7d>:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  t = torch.tensor(t)
<ipython-input-7-8f500bc53f7d>:27: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(sigma**t)


RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cpu and cuda:0! (when checking argument for argument mat1 in method wrapper_CUDA_addmm)